# Table of Contents
* [make gifs](#make-gifs)


In [3]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload

from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import functools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

import copy
import os
import json
from tqdm import tqdm as tqdm

from IPython.display import Image
import PIL.Image as pil
import random
import cv2
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default
#%base16_mplrc dark solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)

from anigen_tools.dataset import FlintstonesDataset
from anigen_tools.mturk import unpickle_this, pickle_this
from anigen_tools.trajectory_viz import filter_chars, filter_description
from anigen_tools.trajectory_viz import avg_video_appearance
from anigen_tools.trajectory_viz import draw_trajectory_set
from anigen_tools.trajectory_viz import vid_filter

In [5]:
from anigen_tools.mturk import unpickle_this
from anigen_tools.docs import doc_video_group
from anigen_tools.segmentation import segment_all_video_entities
from anigen_tools.hybrid_segmentation import draw_video_segmentations
from anigen_tools.tracking import draw_video_tracking

In [4]:
import os

from paramiko import SSHClient
from scp import SCPClient

import paramiko

from scp import SCPClient

In [8]:
# with open('first_sample_ids.txt', 'r') as f:
#     sample_ids = [l.strip() for l in f.readlines()]

In [9]:
sample_ds = unpickle_this('sample_ds_100.pkl')

In [10]:
from tqdm import tqdm

In [8]:
def get_remote_file(remote_dir, name):
    ssh = SSHClient()
    ssh.load_system_host_keys()
    ssh.connect("ec2-54-218-100-182.us-west-2.compute.amazonaws.com", username="ubuntu", key_filename="/Users/schwenk/dustins_crap/wgmm/long_term_store/inter/dustins-dev.pem")
    scp = SCPClient(ssh.get_transport(), sanitize=lambda x: x)
    scp.get(os.path.join(remote_dir, name + "*"), os.path.join('retrieved', remote_dir))

In [11]:
segmentation_dir = 'trajectories/segmentation/'

In [12]:
interp_dir = 'trajectories/interpolation/'
tracking_dir = 'trajectories/tracking/'
frame_dir = 'trajectories/frame_arr_data/'
remote_dirs = [interp_dir, tracking_dir, frame_dir]

In [13]:
cutout_dir = 'trajectories/cutout'

In [70]:
for vid in sample_ids:
    for rd in remote_dirs:
        get_remote_file(rd, vid)

In [52]:
for vid in sample_ids:
    get_remote_file(segmentation_dir, vid)

In [186]:
for vid in sample_ids:
    get_remote_file(cutout_dir, vid)

In [39]:
segment_all_video_entities(pvid, True)

In [19]:
draw_video_segmentations(sample_vids_all[0], True)

# make gifs

In [18]:
for vid in tqdm(sample_ds):
    try:
        draw_video_segmentations(vid, retrieved=True)
    except:
        print(vid.gid())

  0%|          | 0/99 [00:00<?, ?it/s]

./retrieved/trajectories/viz/s_01_e_08_shot_003729_003803_char_0_segm.gif
./retrieved/trajectories/viz/s_01_e_08_shot_003729_003803_char_1_segm.gif
./retrieved/trajectories/viz/s_01_e_08_shot_003729_003803_pool_0_6_segm.gif
./retrieved/trajectories/viz/s_01_e_08_shot_003729_003803_suit_1_6_segm.gif
./retrieved/trajectories/viz/s_01_e_08_shot_003729_003803_chalk_1_13_segm.gif


  1%|          | 1/99 [00:02<04:14,  2.60s/it]

./retrieved/trajectories/viz/s_04_e_17_shot_030776_030850_char_0_segm.gif
./retrieved/trajectories/viz/s_04_e_17_shot_030776_030850_char_1_segm.gif
./retrieved/trajectories/viz/s_04_e_17_shot_030776_030850_eye_shadow_0_16_segm.gif
./retrieved/trajectories/viz/s_04_e_17_shot_030776_030850_vase_1_4_segm.gif


  2%|▏         | 2/99 [00:04<03:34,  2.21s/it]

...

 98%|█████████▊| 97/99 [02:14<00:02,  1.39s/it]

./retrieved/trajectories/viz/s_06_e_07_shot_008468_008542_char_0_segm.gif
./retrieved/trajectories/viz/s_06_e_07_shot_008468_008542_char_1_segm.gif


 99%|█████████▉| 98/99 [02:16<00:01,  1.39s/it]

./retrieved/trajectories/viz/s_06_e_01_shot_018777_018851_char_0_segm.gif


100%|██████████| 99/99 [02:16<00:00,  1.38s/it]


In [17]:
# _ = [draw_video_tracking(vid, retrieved=True) for vid in tqdm(sample_ds)]

100%|██████████| 99/99 [00:13<00:00,  7.14it/s]


In [14]:
local_path = './retrieved/trajectories/improved_cutout/'
keyframe_postfixes = ['_f' + str(x) + '_frame.png' for x in [10, 40, 70]]
def combine_bg_images(video):
    outfile = video.gid() + '_bgfill.png'
    frame_paths = [''.join([local_path, video.gid(), pfix + '_RESULT.png']) for pfix in keyframe_postfixes]
    try:
        frame_images = [pil.open(fp) for fp in frame_paths]
    except FileNotFoundError as e:
        print(e)
        return
    combined_frames = pil.fromarray(np.hstack([np.asarray(img) for img in frame_images if img]))
    combined_frames.save('./retrieved/trajectories/viz/' + outfile)

In [63]:
# _ = [combine_bg_images(vid) for vid in sample_ds]

# make pages

In [41]:
doc_video_group(sample_ds)